In [ ]:
""" get percentage of each flood class in the labeled dataset """

import numpy as np

from datasets.datasets import SN8Dataset

input_csv = ""

dataset = SN8Dataset(input_csv,
                     data_to_load=["preimg","postimg", "building", "road", "roadspeed", "flood"],
                     img_size=(1300,1300))
batch_size=1
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

flood_labels = {0:"non-flooded building", 1:"flooded building", 2:"non-flooded road", 3:"flooded road",4:"background"}
flood_hist = {0:0,1:0,2:0,3:0,4:0}

for i, data in enumerate(dataloader):
    preimg, postimg, building, road, roadspeed, flood = data # data from here is shape (batch_size, channels, height, width)
    # preimg, postimg are shape=(batch_size, 3, h, w)
    # building labels are shape=(batch_size, 1, h, w)
    # road labels are shape=(batch_size, 1, h, w)
    # roadspeed labels are shape=(batch_size, 8, h, w)
    # flood labels are shape=(batch_size, 4, h, w)

    preimg = preimg.cpu().numpy()[0]
    postimg = postimg.cpu().numpy()[0]
    flood = flood.cpu().numpy()[0]
    building = building.cpu().numpy()[0]
    road = road.cpu().numpy()[0]
    roadspeed = roadspeed.cpu().numpy()[0]

    flood_shape = flood.shape # (4, H, W).
    total_pixs = flood.shape[1]*flood_shape[2]
    this_samp_total = 0
    for j in range(len(flood)):
        flood_hist[j] += np.sum(flood[j])
        this_samp_total += np.sum(flood[j])
    flood_hist[4] += total_pixs - this_samp_total
    print(i+1, end="\r")

print()

print("total percentages of foreground class in the dataset (excluding background):")
for i in range(len(flood_hist)-1):
    print(f"  {flood_labels[i]}: {np.round(flood_hist[i] / (flood_hist[0] + flood_hist[1] + flood_hist[2] + flood_hist[3]) * 100, 2)} %")